# Overview
This notebook was created to save and make notes of the fabulous NLP book:  **Handbook Of Natural Language Processing**

# Classifical Approaches
## The classical toolkit
The classical techniques first tackled a nature language by first decomposing the processing process into different steps / stages:
1. tokenization
2. lexical analysis
3. syntactic analysis
4. semantic analysis
5. pragmatic analysis

From the last layer, we proceed to extract the speaker's intended meaning.  

The book dives into each of these $5$ steps

## Text preprocessing
This is defined as the skill of converting text (boiled down to a sequence of bits) into a totally different sequence of meaningful linguistic units. Among the main techniques there is Word Segmentation: dividing a piece of text into different words, this raises the question of what determines the boundaries of a word, sentence segmentation: dividing a piece of text into different sentences. These two techniques are interdependent.  
Text preprocessing should consider different aspect of the language such as the writing system. There are $3$ main system:
1. logographic: a word is built out of a large number of symbols (some can reach $1000$ symbols)
2. syllabic: a symbol represents a syllable (at least two different sounds combined)
3. alphabetic: a symbol represents a sound.  

Due to the cultural exchange taking place in the modern age, no language is purely based on a single writing system. nevertheless, it is safe to state that English is mostly alphabetic.   
### Language identification: The Language dependence Challenge
Well this task does not require a heavy machinery of NLP techniques. The first step is to consider the range of characters in the used language (mainly the mapping of the characters to their numerical values in the encoding system). The first step already narrows down the choice. The second part is a bit trickier as several language could share the same alphabet: Arabic and Persian, Swedish and Danish, european languages in general. This further distinction is mainly based on the distribution of the frequencies of these chracters in the specific set.

### Corpus dependence


Robustness represented a major challenge for the earlier NLP systems. With the explosion of content on the internet, textual data is found in abandance. Nevertheless, the quality did not follow along with the quantity as most piece of text do not follow grammars, punctuations as well as formatting rules that some of the earlier NLP were built upon. It is evident that NLP requires robust algorithms capable of addressing a large number of irregularities associated with 1. difficulty to actually set and formally define rules for producing text 2. the slim likelihood that individuals will actually follow these rules if they ever formalized.

## application dependence
There is no universal criteria determining what constitutes a word, as the possible definitions of such entity can be greatly vary with respect to the language, context, writing system or even the system actually processing the text. One main example is the contraction: I am $\rightarrow$ I'm. Different corpora might treat this linguistic entity differently depending on several factors. Therefore, tokens' representation highly depends on the final processing purpose: speech recognition, classification tasks, text generation... 

## Tokenization 
The challenges associated with segmentation can be explained by two main factors:
1. writing system: I have already elaborated on that above
2. 